In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_columns',None)
#os.chdir("/home/anubrata/Documents/datascience")
os.getcwd()

'/home/anubratadas/Documents/Aarti_Data_Linux/Final_Analysis'

In [2]:
arti_data=pd.read_excel("all_samples.xls")

In [3]:
arti_data.head()

,Position,REF,ALT,stm_1_S13,C_Cip,A_cip,cip_2B_S14,ntrc_gDNA_2,SCV_2,Repl2_244M,Change_type,Coverage,Warnings,Gene_ID,Gene_name,Bio_type,Trancript_ID,Exon_ID,Exon_Rank,Effect,old_AA/new_AA,Old_codon/New_codon,Codon_Num(CDS),Codon_Degeneracy,CDS_size,Codons_around,AAs_around,Custom_interval_ID,Unnamed: 28,Unnamed: 29
0,507,C,T,./.,1/1,1/1,1/1,./.,./.,1/1,SNP,4179.0,gene1,thrA,Transcript_gene1,Exon_NC_003197.2_336_2798,1,SYNONYMOUS_CODING,G/G,ggC/ggT,57,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,537,A,C,./.,1/1,1/1,1/1,./.,./.,1/1,SNP,4690.0,gene1,thrA,Transcript_gene1,Exon_NC_003197.2_336_2798,1,SYNONYMOUS_CODING,A/A,gcA/gcC,67,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,744,A,C,./.,1/1,1/1,1/1,./.,./.,1/1,SNP,5187.0,gene1,thrA,Transcript_gene1,Exon_NC_003197.2_336_2798,1,SYNONYMOUS_CODING,G/G,ggA/ggC,136,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,747,T,C,./.,1/1,1/1,1/1,./.,./.,1/1,SNP,5560.0,gene1,thrA,Transcript_gene1,Exon_NC_003197.2_336_2798,1,SYNONYMOUS_CODING,L/L,ctT/ctC,137,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,762,G,A,./.,1/1,1/1,1/1,./.,./.,1/1,SNP,5371.0,gene1,thrA,Transcript_gene1,Exon_NC_003197.2_336_2798,1,SYNONYMOUS_CODING,G/G,ggG/ggA,142,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
arti_data.groupby('Exon_ID')['Gene_ID'].count()

Exon_ID
1                         3010
CODON_INSERTION              1
FRAME_SHIFT                 11
NON_SYNONYMOUS_CODING     5491
NON_SYNONYMOUS_START         2
START_LOST                   4
STOP_GAINED                 25
STOP_LOST                    9
SYNONYMOUS_CODING        28830
SYNONYMOUS_START             2
SYNONYMOUS_STOP             27
Name: Gene_ID, dtype: int64

In [5]:
rev_df_1=arti_data.iloc[:,[0,10,12,13,14,15,17]]          # select only significant coloumns
cr_plt_dt=rev_df_1[rev_df_1.Change_type=="SNP"]           # select only SNPs as we want to map mutations in genes

In [6]:
cr_plt_dt.head()

,Position,Change_type,Warnings,Gene_ID,Gene_name,Bio_type,Exon_ID
0,507,SNP,gene1,thrA,Transcript_gene1,Exon_NC_003197.2_336_2798,SYNONYMOUS_CODING
1,537,SNP,gene1,thrA,Transcript_gene1,Exon_NC_003197.2_336_2798,SYNONYMOUS_CODING
2,744,SNP,gene1,thrA,Transcript_gene1,Exon_NC_003197.2_336_2798,SYNONYMOUS_CODING
3,747,SNP,gene1,thrA,Transcript_gene1,Exon_NC_003197.2_336_2798,SYNONYMOUS_CODING
4,762,SNP,gene1,thrA,Transcript_gene1,Exon_NC_003197.2_336_2798,SYNONYMOUS_CODING


In [9]:
gene_with_biotype=cr_plt_dt.groupby(['Gene_ID',"Bio_type"])["Gene_ID"].count().sort_values(ascending=False)  
gene_index=gene_with_biotype.index                                                            

In [10]:
gene_with_biotype.head()

Gene_ID        Bio_type                        
Gene_gene2690  Gene_gene2690                       179
STM4261        Exon_NC_003197.2_4482489_4499168    174
STM4489        Exon_NC_003197.2_4731980_4735495     72
metH           Exon_NC_003197.2_4408264_4412034     69
carB           Exon_NC_003197.2_77046_80273         59
Name: Gene_ID, dtype: int64

In [10]:
# function to collect genes and their chromosome
# not used here
def mutation_per_gene():
        lst=[]
        for gene,gene_details in gene_index:
            values=gene_details.split('_')
            if "Exon" in values:
                tup=(gene,values[2],values[3],values[4])
                lst.append(tup)
#print(lst)        

In [54]:
new_data=zip(gene_index,gene_with_biotype.values)

In [55]:
new_data_list=list(new_data)

In [69]:
trial=new_data_list[0:5] # subset of data for initialization

In [74]:
for a,b in trial:
    if "Exon" in a[1]:
        var=a[1].split("_")
        print(var[3],var[4])

4482489 4499168
4731980 4735495
4408264 4412034
77046 80273


In [75]:
# function to collect genes, their chromosome and mutations 
def mutation_df(data_list):
    genn=[] # create empty list for gene names
    chrn=[] # create empty list for chromosome
    gene_start=[] # create empty list for gene start
    gene_end=[] # create empty list for gene end
    mut_num=[] # create empty list for mutations
    for id1,id2 in data_list:
        if "Exon" in id1[1]: # collect only mutations mapped to chromosomes
            genn.append(id1[0])
            chrn.append(id1[1].split('_')[2]) # collect the chromosome ids only using split
            gene_start.append(id1[1].split('_')[3]) # collect the chromosome ids only using split
            gene_end.append(id1[1].split('_')[4]) # collect the chromosome ids only using split
            mut_num.append(id2)
    return chrn,gene_start,gene_end,mut_num,genn                 
           
mutation_data_circos=mutation_df(new_data_list)   

In [76]:
circos_df_dict={"chrm":mutation_data_circos[0],"start":mutation_data_circos[1],"end":mutation_data_circos[2],"mutations":mutation_data_circos[3],"gene":mutation_data_circos[4]}

In [77]:
circos_df=pd.DataFrame.from_dict(circos_df_dict)

In [78]:
circos_df.head()

,chrm,start,end,mutations,gene
0,003197.2,4482489,4499168,174,STM4261
1,003197.2,4731980,4735495,72,STM4489
2,003197.2,4408264,4412034,69,metH
3,003197.2,77046,80273,59,carB
4,003197.2,3494529,3498989,57,gltB


In [79]:
circos_df.groupby("chrm")["gene","mutations"].count()

/home/anubratadas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,gene,mutations
chrm,,
003197.2,3523,3523
003277.2,1,1


In [35]:
rename_chrm=np.where(circos_df.chrm=="003277.2","sal2","sal1")

In [37]:
rename_chrm.shape,circos_df.shape

((3524,), (3524, 3))

In [80]:
circos_df.chrm = np.where(circos_df.chrm=="003277.2","sal2","sal1")

In [81]:
circos_df.head()

,chrm,start,end,mutations,gene
0,sal1,4482489,4499168,174,STM4261
1,sal1,4731980,4735495,72,STM4489
2,sal1,4408264,4412034,69,metH
3,sal1,77046,80273,59,carB
4,sal1,3494529,3498989,57,gltB


In [82]:
circos_df.to_csv("all_samples_mutations.csv",index=False,sep="\t")

In [4]:
cr_plt_dt.Change_type.unique(),cr_plt_dt.Bio_type.unique()

(array(['SNP'], dtype=object),
 array(['Exon_NC_003197.2_336_2798', 'Exon_NC_003197.2_2800_3729',
        'Exon_NC_003197.2_3733_5019', ...,
        'Exon_NC_003197.2_4855387_4856103',
        'Exon_NC_003197.2_4856739_4857425', 'Exon_NC_003277.2_32922_33959'],
       dtype=object))

In [5]:
# function to derive NCBI ids from Bio_type
def get_id(inpt):
    inpt_as_str=str(inpt)
    rev_str_inpt=inpt_as_str[8:16]
    if len(rev_str_inpt)>=0:
        return rev_str_inpt
bio_type=cr_plt_dt.Bio_type.map(get_id)    

In [53]:
bio_type.name="NCBI_ID"

In [37]:
def count_genes_chr(input_dataframe): # count the number of genes for 003197.2 and 003277.2
    unique_ids=cr_plt_dt.Bio_type.unique()
    for item in unique_ids:
        item=str(item)
        nitem=item.split("_")
        if len(nitem)>3:
            z_object=zip(item[2],item[3],item[4])
            print(z_object)
#count_genes_chr(cr_plt_dt)    

In [54]:
cr_plt_dt.shape, bio_type.shape,bio_type.unique()

((42952, 7),
 (42952,),
 array(['003197.2', '', 'pt_gene3', 'pt_gene9', 'pt_gene1', 'pt_gene2',
        'e240', 'e297', 'e313', 'e325', 'pt_gene4', 'pt_gene5', 'e559',
        'pt_gene6', 'pt_gene7', 'e789', 'pt_gene8', 'e1182', 'e1241',
        'e1244', 'e1470', 'e1549', 'e1662', 'e1929', 'e2002', 'e2131',
        'e2210', 'e2665', 'e2690', 'e2975', 'e3302', 'e3460', 'e3520',
        'e3530', 'e3655', 'e3810', 'e3948', 'e4458', '003277.2'],
       dtype=object))

In [55]:
df_circos=pd.concat([cr_plt_dt,bio_type],axis=1)

In [56]:
df_circos.columns

Index(['Position', 'Change_type', 'Warnings', 'Gene_ID', 'Gene_name',
       'Bio_type', 'Exon_ID', 'NCBI_ID'],
      dtype='object')

In [59]:
df_circos[df_circos.NCBI_ID=='e313'] # red flag: this should be intergenic

,Position,Change_type,Warnings,Gene_ID,Gene_name,Bio_type,Exon_ID,NCBI_ID
3448,359230,SNP,Gene_gene313,Gene_gene313,INTRAGENIC:,Gene_gene313,NaN,e313
3449,359236,SNP,Gene_gene313,Gene_gene313,INTRAGENIC:,Gene_gene313,NaN,e313
3450,359384,SNP,Gene_gene313,Gene_gene313,INTRAGENIC:,Gene_gene313,NaN,e313
3451,359529,SNP,Gene_gene313,Gene_gene313,INTRAGENIC:,Gene_gene313,NaN,e313
3452,359542,SNP,Gene_gene313,Gene_gene313,INTRAGENIC:,Gene_gene313,NaN,e313
3453,359603,SNP,Gene_gene313,Gene_gene313,INTRAGENIC:,Gene_gene313,NaN,e313
3454,359757,SNP,Gene_gene313,Gene_gene313,INTRAGENIC:,Gene_gene313,NaN,e313
3455,359839,SNP,Gene_gene313,Gene_gene313,INTRAGENIC:,Gene_gene313,NaN,e313
3456,359976,SNP,Gene_gene313,Gene_gene313,INTRAGENIC:,Gene_gene313,NaN,e313
3457,360078,SNP,Gene_gene313,Gene_gene313,INTRAGENIC:,Gene_gene313,NaN,e313


In [42]:
#cr_plt_dt.Bio_type.map(lambda x:str(x))

In [6]:
non_synonymous_genes=arti_data[arti_data.Exon_ID=="NON_SYNONYMOUS_CODING"]

In [7]:
pd.set_option('display.max_rows',None)
pd.value_counts(non_synonymous_genes.Gene_ID).sort_values(ascending=False).head()

STM4261    64
STM4489    21
ratA       20
slrP       19
STM0019    18
Name: Gene_ID, dtype: int64